#  Enhancing Wi-Fi Networks with AI: A Deep Dive into Machine Learning for Wi-Fi Health Checks




### Introduction
In today's connected world, reliable Wi-Fi is crucial. But keeping it running smoothly can be tricky with all the potential issues like interference and signal drops. We're here to show you how AI, specifically supervised machine learning, can help spot and fix decrease of data ratefor a moving station. We'll walk you through how to build a model that detects performance drops in Wi-Fi networks, making network management more proactive and efficient.

Whether you're a tech expert or just curious about AI, this workshop will give you practical insights and hands-on experience. Let's dive in and explore how machine learning can make Wi-Fi better for everyone!



## Objectives
* Get Familiar with Supervised Learning: Learn the basics of supervised learning, including key concepts, algorithms, and how it's used to make predictions based on labeled data.

* Introduction to Time Series Modeling for Machine Learning: Gain a gentle introduction to time series modeling, exploring how to handle sequential data and apply it in a machine learning context.

* Apply Time Series Modeling to Detect Decreases in Data Rate: Discover practical applications by using time series modeling to identify and address drops in Wi-Fi performance, specifically focusing on detecting decreases in data rate.


## Data Generation

**TODO**: Murat to add the details of how data was generated

### PCAP Extraction process
To extract the content of the PCAP file we used the following command:

```
tshark -o 'gui.column.format:"cus.protocol","%p","Protocol","%p","cus.wlan.fc.type","%Cus:wlan.fc.type:0:R","cus.wlan.fc.type_subtype","%Cus:wlan.fc.type_subtype:0:R","cus.wlan.ssid","%Cus:wlan.ssid:0:R"' \
-T fields \
-e _ws.col.cus.protocol \
-e _ws.col.Protocol \
-e frame.protocols \
-e frame.number \
-e frame.time_epoch \
-e wlan.fc.type \
-e _ws.col.cus.wlan.fc.type \
-e wlan.fc.type_subtype \
-e _ws.col.cus.wlan.fc.type_subtype \
-e wlan.ssid \
-e _ws.col.cus.wlan.ssid \
-e wlan.tag \
-e wlan.bssid \
-e wlan.sa \
-e wlan.da \
-e wlan.ra \
-e wlan.ta \
-e wlan_radio.channel \
-e wlan_radio.signal_dbm \
-e wlan_radio.data_rate \
-e wlan.rsn.capabilities \
-e wlan.rsn.akms.list \
-e wlan.rsn.pcs.list \
-e wlan.qos.tid \
-e radiotap.channel.freq \
-e radiotap.dbm_antsignal \
-e radiotap.datarate \
-e wlan.addr \
-e wlan.seq \
-e wlan.fc.retry \
-e wlan.fc.retry.expert \
-e radiotap.data_retries \
-e wlan.duration \
-e wlan.qbss.cu \
-e radiotap.quality \
-e wlan_radio.signal_percentage \
-e wlan.fcs \
-e wlan.fcs.status \
-e wlan.fc \
-e wlan.fc.moredata \
-e frame.len \
-e wlan_radio.duration \
-e frame.cap_len \
-e wlan.qos.priority \
-e wlan.qos.ack \
-e wlan.qos.buf_state_indicated \
-e wlan_radio.noise_dbm \
-e tcp.analysis.ack_rtt \
-e tcp.analysis.lost_segment \
-e tcp.analysis.retransmission \
-e frame.time_delta \
-e wlan_radio.snr \
-e wlan.fc.more_fragments \
-e wlan.fixed.ssc.fragment \
-e wlan.fragment \
-e wlan.fragment.count \
-e wlan.fragment.error \
-e wlan_radio.frequency \
-e radiotap.present.db_antnoise \
-e radiotap.present.db_antsignal \
-e radiotap.present.dbm_antnoise \
-e radiotap.present.dbm_antsignal \
-E aggregator="$" \
-E separator=/t \
-E header=y \
-r 'wifi_file.pcap' \
-w 'output_file.csv'
```


where `wifi_file.pcap` is the PCAP file and `output.csv` is csv structured output that we will use.

### Verify runtime environment

In [ ]:
try:
    import google.colab
    from google.colab import output        
    output.enable_custom_widget_manager()         # Enable custom widgets in Colab for profile reports
    IN_COLAB = True
    # Load the autoreload extension for IPython
    %load_ext autoreload
    # Set the autoreload extension to reload modules every time they are imported, so that changes made to code in the src folder are reflected in the running code
    %autoreload 2

    %pip install ydata_profiling==4.8.3 \
            scikit-learn==1.5.0 \
            itables==2.1.0 \
            ipywidgets==8.1.2 \
            numpy==1.24.1
        
except:
    IN_COLAB = False

### Import libraries

In [ ]:
import os                       # Interact with the operating system
import pandas as pd             # Data analysis and manipulation
import numpy as np              # Numerical operations on arrays
import matplotlib.pyplot as plt         # Create plots
from itables import show                # To create interactive tables
from ydata_profiling import ProfileReport     # Generate data profile reports
import zipfile

### Reading the data

In [ ]:
%%time
import os
from zipfile import ZipFile
from ipywidgets import interact

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
  # Define the URL of the zip file
  url = "https://github.com/b-yond-infinite-network/sharkfest-us-2024/raw/main/data/wifi-health-checks-data.zip"

  # Define the local file path where the zip file will be saved
  data_path = "/content/wifi-health-checks-data.zip"
  local_extract_path ="/content/tmp"

  # Download the zip file using wget
  !wget -O {data_path} {url}

  # Confirm the download location
  print(f"Zip file downloaded to: {local_zip_path}")
else:
  # Define the path of the zip file
  data_path = "../data/wifi-health-checks-data.zip"
  local_extract_path = "../data/tmp"

# Extract the zip file
with ZipFile(data_path) as zip_file:
    zip_file.extractall(local_extract_path)

# Collect the list of CSV file paths
files = [os.path.join(local_extract_path, file_name) for file_name in os.listdir(local_extract_path) if file_name.endswith('.csv')]

# choose the file from a dropdown list inside the notebook
df = None
def load_file(file):
    """
    Load the file into a dataframe and display the info
    """
    global df
    df = pd.read_csv(file)
    return df.info()

interact(load_file, file=files)

## Functions to preprocess the data
For each file we will:
* remove the empty columns
* change epoch times ('frame_time_epoch') to timestamps
* filter the rows with specific source address and keep only data frames
* index the file by the timestamps

In [ ]:
# function to convert epoch times to datetime
def convert_epoch_to_datetime(df):
    df['timestamps'] = pd.to_datetime(df['frame_time_epoch'], unit='s')
    return df

# function to select frames with 'wlan_fc_type' == 2 and 'wlan_sa' is not 'source_address'
def select_frames(df, source_address):
    return df[(df['wlan_fc_type'] == 2) & (df['wlan_sa'] != source_address)]  # Is this the right way?

# function to remove empty columns
def remove_empty_columns(df):
    return df.dropna(axis=1, how='all')

def preprocess(df, source_address):
    res = convert_epoch_to_datetime(df)
    res = select_frames(res,source_address)
    res = remove_empty_columns(res)
    return res


# Lets apply the preprocessing function to the df we created earlier
source_address = '00:00:00:00:00:02'  #this is the client address. we don't want packets where it is the source address
df = preprocess(df, source_address)
df.info()       # check the column names and their data types

### Some Observations
- The number of columns have been reduced
- The 'timestamps' column is added at the end
- The number of rows reduced

In [ ]:
%%time

# Let us now analyze the data statistically using ydata_profiling library
profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)

# Define the report path
report_path = "./output/report/analysis_report.html"

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(report_path), exist_ok=True)

# we can save the analysis in the drive
profile.to_file(report_path)


### Function to resample the data

The Objective is to resample the data to a specific time interval.

In [ ]:
# create a function which also takes sampling time as input
def resample_file(df, step_size):

    # if the columns exists, set index, if not, it has already been set
    if 'timestamps' in df.columns:
        df.set_index('timestamps', inplace=True)    #set index to timestamps

    #resample data and fill missing values with previous values
    df = df.resample(f'{step_size}s').ffill()

    # with ffil, the first row is NaN, so we drop it
    df = df.dropna(how = 'all')

    return df


# Test the function, resample the data and plot to analyze
step_size = 1
resampled_df = resample_file(df, step_size)

# Let's plot the 'wlan_radio_data_rate' vs timestamps
plt.figure(figsize=(12, 5))
plt.plot(resampled_df.index, resampled_df['wlan_radio_data_rate'])
plt.scatter(resampled_df.index, resampled_df['wlan_radio_data_rate'], color='red', s=10)
plt.xlabel('timestamps')
plt.ylabel('wlan_radio_data_rate')
plt.title('wlan_radio_data_rate vs timestamps')
plt.show()


## Feature Creation
The idea is to create a set of features using the past history of the data. We will use our variable of interest `wlan_radio_datarate` and its past values to create the features.

In [ ]:
# create a function to create features from the 'wlan_radio_data_rate' column
def create_features(df, past_history_sec):

    # convert past_history from seconds to number of samples, based on the resampling time
    past_history_samples = int(past_history_sec / step_size)

    for i in range(1, past_history_samples+1):

        df[f'data_rate_lag_{i}'] = df['wlan_radio_data_rate'].shift(i)

    # remove initial rows with NaN values in the new columns
    df = df.iloc[past_history_samples:]

    return df

# Test the function, create features and plot to analyze
past_history_sec = 7  #s, *times the resampling time

df = create_features(resampled_df, past_history_sec)
show(df)

In [ ]:
%%time

# Let us now analyze the data statistically using ydata_profiling library
profile2 = ProfileReport(df, title='Pandas Profiling Report', explorative=True)

# Define the report path
report_path2 = "./output/report/analysis_report2.html"

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(report_path2), exist_ok=True)

# we can save the analysis in the drive
profile.to_file(report_path2)


# Lets do all of the above steps on all the files in the folder in a loop and then save the results in a big dataframe

In [ ]:
df_complete = pd.DataFrame()

# fix the step size and past history
step_size = 1
past_history = 15

past_history_samples = int(past_history_sec / step_size)

for file in files:
    df = pd.read_csv(file)
    df = preprocess(df, source_address)
    df = resample_file(df, step_size)

    df = create_features(df, past_history)

    df_complete = pd.concat([df_complete, df])

show(df_complete)

## Model Development

<img src="../assets/ml_process.jpg">


### Select features and split the data

In [ ]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# select the 'wlan_radio_data_rate' as features and its lags equal to 'past_history_samples' as features
features = [f'data_rate_lag_{i}' for i in range(1, past_history_samples+1)]
features.append('wlan_radio_data_rate')
# print(features)

target = 'label'

X = df_complete[features]
y = df_complete[target]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=20,
                                                    shuffle=False
                                                    )

#  random forest classifier
rf = RandomForestClassifier(
                            class_weight= 'balanced',
                            random_state=20
                            )
rf.fit(X_train, y_train)

# Predict
y_pred = rf.predict(X_test)

## Evaluation of the model

### Confusion matrix
<img src="../assets/confusion_matrix.png">

### Evaluation Metrics

<img src="../assets/precision_recall.jpg">


In [ ]:
%%time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns

def evaluate_model(y_test, y_predict):

    print("Overall accuracy', %.3f" %accuracy_score(y_test, y_predict))
    precision = precision_score(y_test, y_predict)
    print('Precision: %.3f' % precision)

    recall = recall_score(y_test, y_predict)
    print('Recall: %.3f' % recall)

    f1 = f1_score(y_test, y_predict, average='binary')
    print('F1: %.3f' % f1)

    print("CLASSIFICATION REPORT")
    print(classification_report(y_test, y_predict))

    cm = confusion_matrix(y_test, y_predict)

    labels = np.asarray(
        [ ["{0:0.0f}".format(item) + "\n{0:.2%}".format(item / cm.flatten().sum())]
            for item in cm.flatten() ]
    ).reshape(2, 2)

    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=labels, fmt="", cmap="YlGnBu")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

# call the function
evaluate_model(y_test, y_pred)

## Improving accuracy

The idea is to use the classification threshold that minimizes the difference between precision and recall

In [ ]:
%%time
# Find the best threshold for the model

from sklearn.model_selection import FixedThresholdClassifier

thresholds = np.arange(0.1, 0.95, 0.05)

def find_best_threshold(classifier, X_train, y_train, X_test, y_test, thresholds):

    results = []

    for threshold in thresholds:
        rf_improved = FixedThresholdClassifier(classifier, threshold=threshold)
        y_pred = rf_improved.fit(X_train, y_train).predict(X_test)

        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='binary')

        # if any of the precision or recall is zero, skip the threshold.
        # sklearn sets them to zero when their denominator is zero

        if precision == 0 or recall == 0:
            continue

        results.append([threshold, precision, recall, f1])

    results = pd.DataFrame(results, columns=['threshold', 'precision', 'recall', 'f1_score'])
    results['diff_pre_rec'] = abs(results['precision'] - results['recall'])

    # find the threshold with the minimum difference between precision and recall
    best_threshold = results.loc[results['diff_pre_rec'].idxmin()]

    # train the model with the best threshold and return it
    best_model = FixedThresholdClassifier(classifier, threshold=best_threshold['threshold'])

    return best_threshold, best_model


# Call the function
best_threshold, best_model = find_best_threshold(rf, X_train, y_train, X_test, y_test, thresholds)
print('Best threshold:', best_threshold['threshold'])
best_model.fit(X_train, y_train)

# Evaluate the model with the best threshold
y_pred = best_model.predict(X_test)
evaluate_model(y_test, y_pred)


## Can we further improve the model?

Q1: How often should we make predictions (variable `step_size`)? \
Q2: How many past samples should we use to make predictions (variable `past_history`)?

In [ ]:
%%time
step_size_vals = [1, 2, 3, 5]
past_history_vals = [5, 10, 15, 20, 25]

final_results = []

for step_size in step_size_vals:

    for past_history in past_history_vals:

        print(f'Resampling time: {step_size}, Window size: {past_history}...')

        # if the window size is less than the resampling time, skip the iteration
        if past_history < step_size:
            continue

        past_history_samples = int(past_history / step_size)

        wifi_data = pd.DataFrame()

        for file in files:
            df = pd.read_csv(file)
            df = preprocess(df, source_address)
            df = resample_file(df, step_size)

            df = create_features(df, past_history)

            wifi_data = pd.concat([wifi_data, df])

        # select the wlan_radio_data_rate as features and its lags equal to past_history_samples as features
        features = [f'data_rate_lag_{i}' for i in range(1, past_history_samples+1)]
        features.append('wlan_radio_data_rate')

        X = wifi_data[features]
        y = wifi_data[target]

        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                            test_size=0.2,
                                                            random_state=10,
                                                            shuffle=False
                                                            )

        model = RandomForestClassifier(class_weight= 'balanced',
                                    random_state=20
                                    )
        model.fit(X_train, y_train)

        # finding an optimal threshold value for the model
        best_threshold, best_model = find_best_threshold(model, X_train, y_train, X_test, y_test, thresholds)

        # select the best model metrics and store them
        precision = best_threshold['precision']
        recall = best_threshold['recall']
        f1 = best_threshold['f1_score']

        final_results.append([step_size, past_history, precision, recall, f1, model])

final_results = pd.DataFrame(final_results, columns=['step_size', 'past_history', 'precision', 'recall', 'f1_score', 'model'])


In [ ]:
df_plot = final_results.pivot(index='step_size', columns='past_history', values='f1_score')

plt.figure(figsize=(8, 6))
sns.heatmap(df_plot, annot=True, fmt=".3f", cmap="BuPu")
plt.ylabel("Step size (sec)")
plt.xlabel("Past history (sec)")
plt.title("F1-score (positive class)")
plt.show()

<!-- take examples.  -->


In [ ]:
# choose some positive class indices from the test set and some negative class indices
positive_indices = y_test[y_test == 1].index
negative_indices = y_test[y_test == 0].index

# Create a figure and subplots
fig, axs = plt.subplots(2, 1, figsize=(8, 6))
# Plot positive_indices in red
axs[0].plot(X_test.loc[positive_indices].T, 'r', alpha=0.6)
axs[0].set_title('Features for label 1')
axs[0].set_xticklabels([])

# Plot negative_indices in blue
axs[1].plot(X_test.loc[negative_indices].T, 'b', alpha=0.6)
axs[1].set_title('Features for label 0')
axs[1].set_xticklabels(X_test.columns, rotation=45)

plt.show()


In [ ]:
# # plot feature importance
# importances = best_model.feature_importances_
# indices = np.argsort(importances)[::-1]

# plt.figure(figsize=(6, 4))
# plt.bar(range(X_train.shape[1]), importances[indices])
# plt.xticks(range(X_train.shape[1]), [features[i] for i in indices], rotation=90)
# plt.title("Feature importance")
# plt.show()